In [1]:
# !conda install -y -c conda-forge transformers datasets

In [2]:
# !pip install matplotlib

In [3]:
# !pip install torchsummary

In [4]:
from argparse import ArgumentParser
from datetime import datetime
from typing import Optional
import numpy as np
import pytorch_lightning as pl
import torch
from torch import nn
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader
from torch.utils.data.sampler import Sampler
import torch.nn.functional as F
from transformers import (
    AdamW,
    AutoModel,
    AutoModelForSequenceClassification,
    AutoConfig,
    AutoTokenizer,
    get_linear_schedule_with_warmup,
    glue_compute_metrics,
    PhobertTokenizer
)
import datasets
import pandas as pd
from textprocessor.normalization import VncorenlpTokenizer
from gensim.models import KeyedVectors

from readers.DrQA_Attention import data, model as reader_module, layers
from readers.DrQA_Attention import config as cfg
from readers.DrQA_Attention.lightningmodule import DrQA as DrQA_attn
from readers.DrQA.lightningmodule import DrQA

from pytorch_lightning.loggers import TensorBoardLogger
# from torchsummary import summary

# Prepare Data

In [5]:
class SimpleDataset(torch.utils.data.Dataset):
    def __init__(self, vocab, pos_vocab, ner_vocab, max_seq_length, encodings, aug=None):
        self.encodings = encodings
        self.aug = aug
        self.len_enc = len(self.encodings['labels'])
        self.vocab = vocab
        self.ner_vocab =  ner_vocab
        self.pos_vocab = pos_vocab
        self.max_seq_length = max_seq_length
        if aug is not None:
            self.len_aug = len(self.aug['labels'])
            print('len aug', self.len_aug)
    
    def _single_toksnfeat2ids(self, text_toks:str, pos_toks:str, ner_toks:str):
        try:
            text_toks = text_toks.strip().lower().split(' ')
            pos_toks = pos_toks.strip().lower().split(' ')
            ner_toks = ner_toks.strip().lower().split(' ')
        except Exception:
            print('..........')
            print(text_toks, pos_toks, ner_toks)
        tok_ids = []
        pos_ids = []
        ner_ids = []
        for i in range(len(text_toks)):
            if i >= self.max_seq_length:
                break
            try:
                tok_ids.append(self.vocab[text_toks[i]])
                pos_ids.append(self.pos_vocab[pos_toks[i]])
                ner_ids.append(self.ner_vocab[ner_toks[i]])
            except:
                print('sda..........')
                
                print(text_toks, pos_toks, ner_toks)

        return tok_ids, pos_ids, ner_ids
    
    def idsitem(self, item):
        x1, pos1, ner1 = self._single_toksnfeat2ids(item['x1'], item['x1_f'][0], item['x1_f'][1])
        x2, pos2, ner2 = self._single_toksnfeat2ids(item['x2'], item['x2_f'][0], item['x2_f'][1])
        iitem = {'x1': x1, 'x2': x2, 'x1_f': [pos1, ner1], 'x2_f': [pos2, ner2]}        
        return iitem
    
    def _get_non_aug(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items() if key != 'labels'}
        item = self.idsitem(item)
        item['labels'] = int(self.encodings['labels'][idx])
        return item
    
    def _get_aug(self, idx):
        try:
            item = {key: val[idx] for key, val in self.aug.items() if key != 'labels'}
            item = self.idsitem(item)
            item['labels'] = int(self.aug['labels'][idx])
        except:
            idx = 0
            item = {key: val[idx] for key, val in self.aug.items() if key != 'labels'}
            item = self.idsitem(item)
            item['labels'] = int(self.aug['labels'][idx])
        return item

    def __getitem__(self, idx):
        if self.aug is not None and torch.rand((1, )) < 0.2:
            return self._get_aug(idx % self.len_aug)
        else:
            return self._get_non_aug(idx % self.len_enc)
        return item

    def __len__(self):
        if self.aug is None:
            return self.len_enc
        else:
            return self.len_enc + self.len_aug
            
    def lengths(self):
        ret = [(len(ex[0]), len(ex[1]))
                for ex in zip(self.encodings['x1'], self.encodings['x2'])]
        rm = []
        if self.aug is not None:
            for ex in zip(self.encodings['x1'], self.encodings['x2']):
                try:
                    ret.append((len(ex[0]), len(ex[1])))
                except:
                    continue
        return ret

In [6]:

def _pad(l, val, max_len):
    l += [val]*(max_len - len(l))
    return l
def _pad_group(g):
    max_len = max([len(x) for x in g[0]])
    mask = []
    for i in range(len(g[0])):
        mask.append([0]*len(g[0][i]) + [1]*(max_len - len(g[0][i])))
        _pad(g[0][i], 1, max_len)
        _pad(g[1][i][0], 1, max_len)
        _pad(g[1][i][1], 1, max_len)
#         print(g)
    return mask
def pad(batch):
#     print(batch)
    features = ['x1','x1_f', 'x2', 'x2_f', 'labels']
    d = {f:[] for f in features}
    for i in batch:
        for f in features:
            d[f].append(i[f])
    q = [d['x1'], d['x1_f']]
    p = [d['x2'], d['x2_f']]
    d['x1_mask'] = _pad_group(q)
    d['x2_mask'] = _pad_group(p)
    for k, v in d.items():
        try:
            d[k] = torch.tensor(v)
        except Exception:
            print(v)
    return d

In [7]:
class SortedBatchSampler(Sampler):

    def __init__(self, lengths, batch_size, shuffle=True):
        self.lengths = lengths
        self.batch_size = batch_size
        self.shuffle = shuffle

    def __iter__(self):
        lengths = np.array([(-l[0], -l[1], np.random.random()) for l in self.lengths],dtype=[('l1', np.int_), ('l2', np.int_), ('rand', np.float_)])
        indices = np.argsort(lengths, order=('l1', 'l2', 'rand'))
        batches = [indices[i:i + self.batch_size]
                   for i in range(0, len(indices), self.batch_size)]
        if self.shuffle:
            np.random.shuffle(batches)
        return iter([i for batch in batches for i in batch])

    def __len__(self):
        return len(self.lengths)


class GensimDataModule(pl.LightningDataModule):
    tok_field = 'toks'
    pos_field = 'pos'
    ner_field = 'ner'
    
    q_pref = 'q_'
    t_pref = 't_'
    num_labels = 2

    def __init__(
        self,
        vocab,
        pos_vocab,
        ner_vocab,
        max_seq_length: int = 128,
        train_batch_size: int = 32,
        eval_batch_size: int = 32,
        **kwargs
    ):
        super().__init__()
        self.vocab = vocab
        self.pos_vocab = pos_vocab
        self.ner_vocab = ner_vocab
        self.max_seq_length = max_seq_length
        self.train_batch_size = train_batch_size
        self.eval_batch_size = eval_batch_size

    def filtnan(self, df):
        tmp = df.replace(np.nan, '', regex=True)
        return tmp[(tmp['t_toks'] != '') & (tmp['q_toks'] != '')]
    
    def setup(self, stage, df_ls):
        
        train_df = df_ls[0]
        
        valid_df = df_ls[1]
        if len(df_ls) == 3:
            aug_df = df_ls[2]
            self.dataset = {
                "train": SimpleDataset(self.vocab, self.pos_vocab, self.ner_vocab, self.max_seq_length, self.convert_to_features(train_df), self.convert_to_features(aug_df)),
                "validation": SimpleDataset(self.vocab, self.pos_vocab, self.ner_vocab, self.max_seq_length, self.convert_to_features(valid_df)),
                }
        else:
            self.dataset = {
                "train": SimpleDataset(self.vocab, self.pos_vocab, self.ner_vocab, self.max_seq_length, self.convert_to_features(train_df)),
                "validation": SimpleDataset(self.vocab, self.pos_vocab, self.ner_vocab, self.max_seq_length, self.convert_to_features(valid_df)),
                }
        self.eval_splits = ["validation"]
    def prepare_data(self):
        pass
    def train_dataloader(self):
        sampler = SortedBatchSampler(
            self.dataset['train'].lengths(),
            self.train_batch_size,
            shuffle=False
        )
        return DataLoader(self.dataset['train'], batch_size=self.train_batch_size, collate_fn=pad, sampler=sampler, num_workers=4, pin_memory=True)
    def val_dataloader(self):
        sampler = SortedBatchSampler(
            self.dataset['validation'].lengths(),
            self.eval_batch_size,
            shuffle=False
        )
        return DataLoader(self.dataset['validation'], batch_size=self.eval_batch_size, collate_fn=pad, sampler=sampler, num_workers=4, pin_memory=True)

    def test_dataloader(self):
        return DataLoader(self.dataset['validation'], batch_size=self.eval_batch_size, collate_fn=pad,num_workers=4, pin_memory=True)
    
    def tok2id(self, tokens):
        return tensor([self.vocab[tok.lower()] for tok in tokens], dtype=torch.int64)
    def ids2toks(self, ids):
        return ' '.join([self.vocab[ind] for ind in ids.tolist()])
    
    def _single_toksnfeat2ids(self, text_toks:str, pos_toks:str, ner_toks:str):
        text_toks = text_toks.strip().lower().split(' ')
        pos_toks = pos_toks.strip().lower().split(' ')
        ner_toks = ner_toks.strip().lower().split(' ')
        tok_ids = []
        pos_ids = []
        ner_ids = []
        for i in range(len(text_toks)):
            if i >= self.max_seq_length:
                break
            tok_ids.append(self.vocab[text_toks[i]])
            pos_ids.append(self.pos_vocab[pos_toks[i]])
            ner_ids.append(self.ner_vocab[ner_toks[i]])
        return tok_ids, pos_ids, ner_ids

    
    def toksnfeat2ids(self, tok, pos, ner, padding=True):
        assert len(tok) == len(pos) and len(tok) == len(ner)
        longest = 0
        batch_tok_ids = []
        batch_pos_ids = []
        batch_ner_ids = []
        batch_mask = []
        for i in range(len(tok)):
            try:
                if len(tok[i]) <= 0:
                    continue
                tok_ids, pos_ids, ner_ids = self._single_toksnfeat2ids(tok[i], pos[i], ner[i])
                batch_tok_ids.append(tok_ids)
                batch_pos_ids.append(pos_ids)
                batch_ner_ids.append(ner_ids)
                batch_mask.append([0]*len(tok_ids))
            except TypeError:
#                 print('Skip ', tok[i])
                continue
        return batch_tok_ids, batch_pos_ids, batch_ner_ids, batch_mask
        
    def convert_to_features(self, example_batch, indices=None):
        features = {
            'x1': example_batch[self.q_pref + self.tok_field].tolist(),
            'x1_f': list(zip(example_batch[self.q_pref + self.pos_field], example_batch[self.q_pref + self.ner_field])),
            'x2': example_batch[self.t_pref + self.tok_field].tolist(),
            'x2_f': list(zip(example_batch[self.t_pref + self.pos_field], example_batch[self.t_pref + self.ner_field])),
            'labels': example_batch['label']
        }
        return features

In [8]:
wv_path = 'wordembedding/pretrain_models/word2vec/baomoi.window2.vn.model.bin'
wv = KeyedVectors.load_word2vec_format(wv_path,binary=True)
vocab = data.W2VVocab(['<unk>', '<pad>', '<cls>'], 0, 1, wv)

In [9]:
ner_vocab = data.ReaderVocab.from_file('dataset/vocabs/train_feat_ner_vocab.txt')
ner_vocab.UNK='<unk>'
pos_vocab = data.ReaderVocab.from_file('dataset/vocabs/train_feat_pos_vocab.txt')
pos_vocab.UNK='<unk>'

In [10]:
w_in_train = []
with open('dataset/vocabs/zlqa_token_list.txt', 'r') as f:
    while True:
        line = f.readline()
        if not line:
            break
        w_in_train.append(line.split(' ')[-1][:-1].lower())

# Model

# Training

In [11]:
def parse_args(args=None):
    parser = ArgumentParser()
    parser = pl.Trainer.add_argparse_args(parser)
    parser = GensimDataModule.add_argparse_args(parser)
    parser = DrQA.add_model_specific_args(parser)
    parser.add_argument('--seed', type=int, default=42)
    return parser.parse_args(args)


In [12]:
args_normal = cfg.Args()
# args.max_len = 256
args_normal.hidden_size = 512
args_normal.context_layers = 3
args_normal.pre_context_layers = 2
args_normal.question_layers = 2
args_normal.num_encodes = 3
args_normal.num_heads = 6
args_normal.concat_rnn_layers = True
args_normal.dropout_rnn = 0.2
args_normal.dropout_emb = 0.1
args_normal.layernorm_emb = True
args_normal.dropout_rnn_output = True
args_normal.embedding_dim = 300
args_normal.num_features = 2

In [13]:
args = cfg.Args()
args.hidden_size = 512
args.context_layers = 1
args.pre_context_layers = 1
args.question_layers = 2
args.num_encodes = 4
args.num_heads = 8
args.concat_rnn_layers = True
args.dropout_rnn = 0.2
args.dropout_emb = 0.1
args.layernorm_emb = True
args.dropout_rnn_output = True
args.embedding_dim = 300
args.num_features = 2


In [14]:
mocked_args = """
    --max_epochs 5
    --gpus 1
    --learning_rate 2e-3
    --adam_epsilon 1e-8
    --weight_decay 1e-1
    --warmup_steps 200
    --max_seq_length 256
    --train_batch_size 16
    --eval_batch_size 16""".split()

In [15]:
margs = parse_args(mocked_args)
pl.seed_everything(margs.seed)

42

In [16]:
train_df = pd.read_csv('dataset/qaset/train_feature_set2.csv').sample(frac=1).reset_index(drop=True)
aug_df = pd.read_csv('dataset/qaset/aug_feature_set2.csv').sample(frac=1).reset_index(drop=True)
valid_df = pd.read_csv('dataset/qaset/valid_feature_set2.csv').sample(frac=1).reset_index(drop=True)

In [17]:
dm = GensimDataModule.from_argparse_args(margs,vocab=vocab,pos_vocab=pos_vocab, ner_vocab=ner_vocab)
dm.prepare_data()
dm.setup('fit', [train_df, valid_df, aug_df])
# dm.setup('fit', [train_df, valid_df])

len aug 105492


In [18]:
len(dm.train_dataloader())

1811

In [19]:
b = next(iter(dm.train_dataloader()))

In [20]:
print(b['x1'].size())
print(b['x1_f'].size())
print(b['x1_mask'].size())

print(b['x2'].size())
print(b['x2_f'].size())
print(b['x2_mask'].size())

torch.Size([16, 29])
torch.Size([16, 2, 29])
torch.Size([16, 29])
torch.Size([16, 110])
torch.Size([16, 2, 110])
torch.Size([16, 110])


In [21]:
model = DrQA_attn(args, vocab, [len(pos_vocab), len(ner_vocab)], **vars(margs))
model.embedding.load_embeddings(w_in_train, wv)

Loaded 24631 embeddings (100.00%)


In [22]:
print(model)

DrQA(
  (embedding): EmbeddingModule(
    (embedding): Embedding(439057, 300, padding_idx=1)
    (layernorm): LayerNorm((300,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (embedding_feature): ModuleList(
    (0): Embedding(29, 1, padding_idx=1)
    (1): Embedding(12, 1, padding_idx=1)
  )
  (network): RnnDocReader(
    (rnn): CustomRNN(
      (rnns): ModuleList(
        (0): RNNBlock(
          (rnn): LSTM(300, 512)
          (dropout): Dropout(p=0.2, inplace=False)
        )
      )
      (dropout): Dropout(p=0.2, inplace=False)
    )
    (encodes): ModuleList(
      (0): EncodeModule(
        (linear1): Linear(in_features=514, out_features=1028, bias=True)
        (linear2): Linear(in_features=1028, out_features=514, bias=True)
        (norm1): LayerNorm((514,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((514,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (mul_head): Multi

In [23]:
# summary(model, (32, 256, args.embedding_dim))

In [24]:
# model(**next(iter(dm.train_dataloader())))

In [25]:
logger = TensorBoardLogger('tb_logs', name='DrQA_Atten')

In [26]:
# logger.experiment.add_graph(model,[b['x1'], b['x1_f'], b['x1_mask'],b['x2'], b['x2_f'], b['x2_mask']])

In [27]:
trainer = pl.Trainer.from_argparse_args(margs, logger=logger, auto_lr_find=True)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [28]:
# # # Run learning rate finder
# lr_finder = trainer.tuner.lr_find(model, train_dataloader=dm.train_dataloader())
# fig = lr_finder.plot(suggest=True)
# fig.show()

In [29]:
model.hparams.lr = 1e-3 #lr_finder.suggestion()

In [30]:
# model.train_dataloader()

/opt/conda/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: `train_dataloader` must be implemented to be used with the Lightning Trainer
  warnings.warn(*args, **kwargs)


In [31]:
trainer.max_epochs = 3
trainer.fit(model, dm)


  | Name              | Type             | Params
-------------------------------------------------------
0 | embedding         | EmbeddingModule  | 131 M 
1 | embedding_feature | ModuleList       | 41    
2 | network           | RnnDocReader     | 68.9 M
3 | loss_fct          | CrossEntropyLoss | 0     


/opt/conda/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 7.92 GiB total capacity; 6.58 GiB already allocated; 38.69 MiB free; 6.76 GiB reserved in total by PyTorch)

In [ ]:
lr_finder = trainer.tuner.lr_find(model, train_dataloader=dm.train_dataloader())
model.hparams.lr = min(1e-5, lr_finder.suggestion())
trainer.fit(model, dm)

In [ ]:
lr_finder = trainer.tuner.lr_find(model, train_dataloader=dm.train_dataloader())
model.hparams.lr = min(1e-5, lr_finder.suggestion())
trainer.fit(model, dm)

In [31]:
for param in model.embedding.parameters():
    param.requires_grad = False